### Importa Bibliotecas

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import random
import string

### Importa os dados

In [2]:

# Carregar o dataframe com as letras de músicas (substitua isso pelo seu código de carregamento)
df = pd.read_csv('letras_musicas.csv')

df = df[~df.index.isin([375,412])] # drop letras em português
df.reset_index(inplace=True)

df = df[['Letra']]

# Função para quebrar o texto e adicionar novas linhas ao DataFrame
def quebrar_e_adicionar_linhas(row):
    textos_quebrados = row['Letra'].splitlines()
    return pd.DataFrame({'Letra': textos_quebrados})

# Aplicar a função para cada linha do DataFrame e combinar os resultados
novo_df = pd.concat([quebrar_e_adicionar_linhas(row) for _, row in df.iterrows()], ignore_index=True)

novo_df

,Letra
0,
1,"Uhh, uhh, uhh"
2,"You ready, B?"
3,Let's go get 'em
4,Look for me!
...,...
25732,"The people, the people"
25733,"If you ask me, baby"
25734,"I'll do anything you say you, boy"
25735,I’m talking to you


In [3]:
# Pré-processamento das letras
all_lyrics = " ".join(novo_df['Letra'])  # Concatenar todas as letras
all_lyrics = all_lyrics.lower()  # Converter para minúsculas

In [4]:
all_lyrics

'  uhh, uhh, uhh you ready, b? let\'s go get \'em look for me! young, b, cruisin\' down the westside highway doin\' what we like to do, our way eyes behind shades this necklace the reason all of my dates been blind dates but today i got my thoroughest girl with me i\'m mashin\' the gas, she\'s grabbin\' the wheel, it\'s trippy how hard she rides with me, the new bobby and whitney only time we don\'t speak is during sex and the city she gets carrie fever, but soon as the show\'s over she\'s right back to being my soldier ‘cause mami\'s a rider and i\'m a roller put us together, how they gon\' stop both us? whatever she lacks i\'m right over her shoulder when i\'m off track, mami is keepin\' me focused so let\'s lock this down like it\'s supposed to be the \'03 bonnie and clyde: hov\' and b all i need in this life of sin is me and my girlfriend (me and my girlfriend) down to ride \'til the very end, is me and my boyfriend (me and my boyfriend) all i need in this life of sin is me and my 

### Constrói Modelo

In [5]:
# Criar um dicionário de caracteres únicos presentes nas letras
chars = sorted(list(set(all_lyrics)))
char_to_index = {char: index for index, char in enumerate(chars)}
index_to_char = {index: char for index, char in enumerate(chars)}

# Parâmetros do modelo
input_size = len(chars)
hidden_size = 128
output_size = len(chars)
num_layers = 2
seq_length = 100  # Tamanho da sequência de entrada

# Definir o modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# Carregar modelo treinado (ou treinar o modelo aqui)
# model.load_state_dict(torch.load('modelo.pth'))

In [6]:
# Função para gerar letras
def generate_lyrics(start_text, num_verses=3, max_length=1000):
    model.eval()
    
    start_text = start_text.lower()
    start_text = start_text[:seq_length]
    
    input_sequence = [char_to_index[char] for char in start_text]
    input_sequence = torch.unsqueeze(torch.tensor(input_sequence), 0)
    
    hidden = (torch.zeros(num_layers, 1, hidden_size),
              torch.zeros(num_layers, 1, hidden_size))
    
    generated_text = start_text
    
    for _ in range(max_length):
        output, hidden = model(input_sequence, hidden)
        probabilities = torch.nn.functional.softmax(output[0][-1], dim=0).numpy()
        predicted_index = random.choices(range(output_size), weights=probabilities)[0]
        predicted_char = index_to_char[predicted_index]
        
        generated_text += predicted_char
        input_sequence = torch.unsqueeze(torch.unsqueeze(torch.tensor([[predicted_index]]), 0), 0)
        
        if predicted_char == '\n':
            num_verses -= 1
            if num_verses <= 0:
                break
    
    return generated_text

### Testa

In [7]:
# Gerar letras
start_text = "i love you"
generated_lyrics = generate_lyrics(start_text, num_verses=3)
print(generated_lyrics)

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
torch.__version__

'2.0.1+cu117'